In [1]:
import cv2
import numpy as np
#from google.colab.patches import cv2_imshow
import re

MODEL_LOCATION = "/dcs/large/u1901447/models/"

In [2]:
# Gets a set length extract of a video file starting from a given frame
def getVideoFrames(filepath,start,length):
  cap = cv2.VideoCapture(filepath)
  width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH ))
  height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT ))
  fps = int(cap.get(cv2.CAP_PROP_FPS))
  cap.set(cv2.CAP_PROP_POS_FRAMES, start-1)
  frames_list=list()
  for frame_count in range(start,start+length):
    ret, frame = cap.read()
    frames_list.append(frame)
  cap.release()
  return frames_list, (width,height), fps

In [3]:
# Parse the label file into list of labels and run lengths
videoLocation = "/dcs/large/u1901447/videos/220611galleivnor_2_movie-001.mov"
labelLocation = "/dcs/large/u1901447/labels/galleinvorLabels.lbl"
with open(labelLocation) as f:
    lines = f.readlines()

label_tuples = list()
for line in lines:
  #print(line)
  label_match = re.search(r'(?<=Label.)(.*)(?=:)',line)
  #print(label_match.group())
  frame_match = re.search(r'(?<=:)(.*)(?=\n)',line)
  if frame_match is None:
    frame_match = re.search(r'(?<=:)(.*)',line)
  #print(frame_match.group())
  label_tuples.append((label_match.group(),int(frame_match.group())))

In [4]:
# Define the location of the video we are processing
filepath = videoLocation
# Define the folder we want to save the extracted fragments into
save_path = '/dcs/large/u1901447/videos/videoClips/'
# Setup a dict to store the counts of files of each label so we can have a unique file name for each typoe of label
label_indexer = dict()
all_labels = set(l[0] for l in label_tuples)
label_indexer = {l:0 for l in all_labels}
frame_counter = 1
# Loop through the RLE data calling the frame extractor function, then saving the extracted froms to a new MP4 file
for label,frames_count in label_tuples:
  # print(label)
  # print(frames_count)
  # we need at least 15 frames to make a decent clip
  if frames_count > 14:
    frames, videodims, fps = getVideoFrames(filepath,frame_counter,frames_count)
    # print(len(frames))
    # print(videodims)
    # print(fps)
    videocap_name = f"{save_path}{label}_{label_indexer[label]}.mp4"
    # increment the indexer for this labels
    label_indexer[label]+=1
    out = cv2.VideoWriter(videocap_name, cv2.VideoWriter_fourcc(*'mp4v'), fps, videodims)
    for frame in frames:
        out.write(frame)
    out.release()
    print(f"Saved filename {videocap_name} with label {label} and {frames_count} frames from {frame_counter} to {frame_counter+frames_count}")
  else:
    print(f"Dropped label a run of {label} with {frames_count} frames from {frame_counter} to {frame_counter+frames_count}")
  # Step the frame counter on to be ready for the next read
  frame_counter += frames_count
  


Saved filename /dcs/large/u1901447/videos/videoClips/NOTHING_0.mp4 with label NOTHING and 249 frames from 1 to 250
Saved filename /dcs/large/u1901447/videos/videoClips/KICK_R_0.mp4 with label KICK_R and 56 frames from 250 to 306
Saved filename /dcs/large/u1901447/videos/videoClips/NOTHING_1.mp4 with label NOTHING and 67 frames from 306 to 373
Saved filename /dcs/large/u1901447/videos/videoClips/TACKLE_S_0.mp4 with label TACKLE_S and 55 frames from 373 to 428
Saved filename /dcs/large/u1901447/videos/videoClips/RUCK_0.mp4 with label RUCK and 71 frames from 428 to 499
Dropped label a run of PASS_L with 14 frames from 499 to 513
Saved filename /dcs/large/u1901447/videos/videoClips/CARRY_0.mp4 with label CARRY and 19 frames from 513 to 532
Saved filename /dcs/large/u1901447/videos/videoClips/TACKLE_S_1.mp4 with label TACKLE_S and 25 frames from 532 to 557
Saved filename /dcs/large/u1901447/videos/videoClips/RUCK_1.mp4 with label RUCK and 231 frames from 557 to 788
Dropped label a run of CA

In [5]:
# Changed to account for changes to keras structures
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import Sequence, img_to_array

2023-02-27 13:05:34.518918: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-27 13:05:36.141556: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /dcs/19/u1901447/year4/project/ruck_and_roll/venv/lib/python3.9/site-packages/cv2/../../lib64:/local/java/postgresql/lib/:/local/java/postgresql/lib/
2023-02-27 13:05:36.141585: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-27 13:05:39.738493: W tensorflow/compiler/xla/stream_executor/platfo

In [6]:
# Rewrite of the required pip import which no longer workd because of changes in keras
"""
VideoFrameGenerator - Simple Generator
--------------------------------------
A simple frame generator that takes distributed frames from
videos. It is useful for videos that are scaled from frame 0 to end
and that have no noise frames.
"""

import glob
import logging
import os
import re
from math import floor
from typing import Iterable, Optional

import cv2 as cv
import numpy as np


log = logging.getLogger()

class VideoFrameGenerator(Sequence):  # pylint: disable=too-many-instance-attributes
    """
    Create a generator that return batches of frames from video
    - rescale: float fraction to rescale pixel data (commonly 1/255.)
    - nb_frames: int, number of frames to return for each sequence
    - classes: list of str, classes to infer
    - batch_size: int, batch size for each loop
    - use_frame_cache: bool, use frame cache (may take a lot of memory for \
        large dataset)
    - shape: tuple, target size of the frames
    - shuffle: bool, randomize files
    - transformation: ImageDataGenerator with transformations
    - split: float, factor to split files and validation
    - nb_channel: int, 1 or 3, to get grayscaled or RGB images
    - glob_pattern: string, directory path with '{classname}' inside that \
        will be replaced by one of the class list
    - use_header: bool, default to True to use video header to read the \
        frame count if possible

    - seed: int, default to None, keep the seed value for split

    You may use the "classes" property to retrieve the class list afterward.
    The generator has that properties initialized:
    - classes_count: number of classes that the generator manages
    - files_count: number of video that the generator can provides
    - classes: the given class list
    - files: the full file list that the generator will use, this \
        is usefull if you want to remove some files that should not be \
        used by the generator.
    """

    def __init__(  # pylint: disable=too-many-statements,too-many-locals,too-many-branches,too-many-arguments
        self,
        rescale: float = 1 / 255.0,
        nb_frames: int = 5,
        classes: list = None,
        batch_size: int = 16,
        use_frame_cache: bool = False,
        target_shape: tuple = (224, 224),
        shuffle: bool = True,
        transformation: Optional[ImageDataGenerator] = None,
        split_test: float = None,
        split_val: float = None,
        nb_channel: int = 3,
        glob_pattern: str = "./videos/{classname}/*.avi",
        use_headers: bool = True,
        seed=None,
        **kwargs,
    ):

        self.glob_pattern = glob_pattern

        # should be only RGB or Grayscale
        assert nb_channel in (1, 3)

        if classes is None:
            classes = self._discover_classes()

        # we should have classes
        if len(classes) == 0:
            log.warn(
                "You didn't provide classes list or "
                "we were not able to discover them from "
                "your pattern.\n"
                "Please check if the path is OK, and if the glob "
                "pattern is correct.\n"
                "See https://docs.python.org/3/library/glob.html"
            )

        # shape size should be 2
        assert len(target_shape) == 2

        # split factor should be a propoer value
        if split_val is not None:
            assert 0.0 < split_val < 1.0

        if split_test is not None:
            assert 0.0 < split_test < 1.0

        self.use_video_header = use_headers

        # then we don't need None anymore
        split_val = split_val if split_val is not None else 0.0
        split_test = split_test if split_test is not None else 0.0

        # be sure that classes are well ordered
        classes.sort()

        self.rescale = rescale
        self.classes = classes
        self.batch_size = batch_size
        self.nbframe = nb_frames
        self.shuffle = shuffle
        self.target_shape = target_shape
        self.nb_channel = nb_channel
        self.transformation = transformation
        self.use_frame_cache = use_frame_cache

        self._random_trans = []
        self.__frame_cache = {}
        self.files = []
        self.validation = []
        self.test = []

        _validation_data = kwargs.get("_validation_data", None)
        _test_data = kwargs.get("_test_data", None)
        np.random.seed(seed)

        if _validation_data is not None:
            # we only need to set files here
            self.files = _validation_data

        elif _test_data is not None:
            # we only need to set files here
            self.files = _test_data
        else:
            self.__split_from_vals(
                split_val, split_test, classes, shuffle, glob_pattern
            )

        # build indexes
        self.files_count = len(self.files)
        self.indexes = np.arange(self.files_count)
        self.classes_count = len(classes)

        # to initialize transformations and shuffle indices
        if "no_epoch_at_init" not in kwargs:
            self.on_epoch_end()

        kind = "train"
        if _validation_data is not None:
            kind = "validation"
        elif _test_data is not None:
            kind = "test"

        self._current = 0
        self._framecounters = {}
        print(
            "Total data: %d classes for %d files for %s"
            % (self.classes_count, self.files_count, kind)
        )

    def count_frames(self, cap, name, force_no_headers=False):
        """Count number of frame for video
        if it's not possible with headers"""
        if not force_no_headers and name in self._framecounters:
            return self._framecounters[name]

        total = cap.get(cv.CAP_PROP_FRAME_COUNT)

        if force_no_headers or total < 0:
            # headers not ok
            total = 0
            # TODO: we're unable to use CAP_PROP_POS_FRAME here
            # so we open a new capture to not change the
            # pointer position of "cap"
            capture = cv.VideoCapture(name)
            while True:
                grabbed, _ = capture.read()
                if not grabbed:
                    # rewind and stop
                    break
                total += 1

        # keep the result
        self._framecounters[name] = total

        return total

    def __split_from_vals(self, split_val, split_test, classes, shuffle, glob_pattern):
        """ Split validation and test set """

        if split_val == 0 or split_test == 0:
            # no splitting, do the simplest thing
            for cls in classes:
                self.files += glob.glob(glob_pattern.format(classname=cls))
            return

        # else, there is some split to do
        for cls in classes:
            files = glob.glob(glob_pattern.format(classname=cls))
            nbval = 0
            nbtest = 0
            info = []

            # generate validation and test indexes
            indexes = np.arange(len(files))

            if shuffle:
                np.random.shuffle(indexes)

            nbtrain = 0
            if 0.0 < split_val < 1.0:
                nbval = int(split_val * len(files))
                nbtrain = len(files) - nbval

                # get some sample for validation_data
                val = np.random.permutation(indexes)[:nbval]

                # remove validation from train
                indexes = np.array([i for i in indexes if i not in val])
                self.validation += [files[i] for i in val]
                info.append("validation count: %d" % nbval)

            if 0.0 < split_test < 1.0:
                nbtest = int(split_test * nbtrain)
                nbtrain = len(files) - nbval - nbtest

                # get some sample for test_data
                val_test = np.random.permutation(indexes)[:nbtest]

                # remove test from train
                indexes = np.array([i for i in indexes if i not in val_test])
                self.test += [files[i] for i in val_test]
                info.append("test count: %d" % nbtest)

            # and now, make the file list
            self.files += [files[i] for i in indexes]
            print("class %s, %s, train count: %d" % (cls, ", ".join(info), nbtrain))

    def _discover_classes(self):
        pattern = os.path.realpath(self.glob_pattern)
        pattern = re.escape(pattern)
        pattern = pattern.replace("\\{classname\\}", "(.*?)")
        pattern = pattern.replace("\\*", ".*")

        files = glob.glob(self.glob_pattern.replace("{classname}", "*"))
        classes = set()
        for filename in files:
            filename = os.path.realpath(filename)
            classname = re.findall(pattern, filename)[0]
            classes.add(classname)

        return list(classes)

    def next(self):
        """ Return next element"""
        elem = self[self._current]
        self._current += 1
        if self._current == len(self):
            self._current = 0
            self.on_epoch_end()

        return elem

    def get_validation_generator(self):
        """ Return the validation generator if you've provided split factor """
        return self.__class__(
            nb_frames=self.nbframe,
            nb_channel=self.nb_channel,
            target_shape=self.target_shape,
            classes=self.classes,
            batch_size=self.batch_size,
            shuffle=self.shuffle,
            rescale=self.rescale,
            glob_pattern=self.glob_pattern,
            use_headers=self.use_video_header,
            _validation_data=self.validation,
        )

    def get_test_generator(self):
        """ Return the validation generator if you've provided split factor """
        return self.__class__(
            nb_frames=self.nbframe,
            nb_channel=self.nb_channel,
            target_shape=self.target_shape,
            classes=self.classes,
            batch_size=self.batch_size,
            shuffle=self.shuffle,
            rescale=self.rescale,
            glob_pattern=self.glob_pattern,
            use_headers=self.use_video_header,
            _test_data=self.test,
        )

    def on_epoch_end(self):
        """ Called by Keras after each epoch """

        if self.transformation is not None:
            self._random_trans = []
            for _ in range(self.files_count):
                self._random_trans.append(
                    self.transformation.get_random_transform(self.target_shape)
                )

        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __iter__(self):
        return self

    def __next__(self):
        return self.next()

    def __len__(self):
        return int(np.floor(self.files_count / self.batch_size))

    def __getitem__(self, index):
        classes = self.classes
        shape = self.target_shape
        nbframe = self.nbframe

        labels = []
        images = []

        indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]

        transformation = None

        for i in indexes:

            video = self.files[i]
            classname = self._get_classname(video)

            # create a label array and set 1 to the right column
            label = np.zeros(len(classes))
            col = classes.index(classname)
            label[col] = 1.0

            if video not in self.__frame_cache:
                frames = self._get_frames(
                    video, nbframe, shape, force_no_headers=not self.use_video_header
                )
                if frames is None:
                    # avoid failure, nevermind that video...
                    continue

                # add to cache
                if self.use_frame_cache:
                    self.__frame_cache[video] = frames

            else:
                frames = self.__frame_cache[video]

            # apply transformation
            # if provided
            if self.transformation is not None:
                transformation = self._random_trans[i]
                frames = [
                    self.transformation.apply_transform(frame, transformation)
                    if transformation is not None
                    else frame
                    for frame in frames
                ]

            # add the sequence in batch
            images.append(frames)
            labels.append(label)

        return np.array(images), np.array(labels)

    def _get_classname(self, video: str) -> str:
        """ Find classname from video filename following the pattern """

        # work with real path
        video = os.path.realpath(video)
        pattern = os.path.realpath(self.glob_pattern)

        # remove special regexp chars
        pattern = re.escape(pattern)

        # get back "*" to make it ".*" in regexp
        pattern = pattern.replace("\\*", ".*")

        # use {classname} as a capture
        # pattern = pattern.replace("\\{classname\\}", "(.*?)")
        # replacement to make it work with _ filenames
        pattern = pattern.replace("\\{classname\\}", "(.*?)_")

        # and find all occurence
        classname = re.findall(pattern, video)[0]

        return classname

    def _get_frames(
        self, video, nbframe, shape, force_no_headers=False
    ) -> Optional[Iterable]:
        cap = cv.VideoCapture(video)
        total_frames = self.count_frames(cap, video, force_no_headers)
        orig_total = total_frames

        if total_frames % 2 != 0:
            total_frames += 1

        frame_step = floor(total_frames / (nbframe - 1))
        # TODO: fix that, a tiny video can have a frame_step that is
        # under 1
        frame_step = max(1, frame_step)
        frames = []
        frame_i = 0

        while True:
            grabbed, frame = cap.read()
            if not grabbed:
                break

            self.__add_and_convert_frame(
                frame, frame_i, frames, orig_total, shape, frame_step
            )

            if len(frames) == nbframe:
                break

        cap.release()

        if not force_no_headers and len(frames) != nbframe:
            # There is a problem here
            # That means that frame count in header is wrong or broken,
            # so we need to force the full read of video to get the right
            # frame counter
            return self._get_frames(video, nbframe, shape, force_no_headers=True)

        if force_no_headers and len(frames) != nbframe:
            # and if we really couldn't find the real frame counter
            # so we return None. Sorry, nothing can be done...
            log.error(
                f"Frame count is not OK for video {video}, "
                f"{total_frames} total, {len(frames)} extracted"
            )
            return None

        return np.array(frames)

    def __add_and_convert_frame(  # pylint: disable=too-many-arguments
        self, frame, frame_i, frames, orig_total, shape, frame_step
    ):
        frame_i += 1
        if frame_i in (1, orig_total) or frame_i % frame_step == 0:
            # resize
            frame = cv.resize(frame, shape)

            # use RGB or Grayscale ?
            frame = (
                cv.cvtColor(frame, cv.COLOR_BGR2RGB)
                if self.nb_channel == 3
                else cv.cvtColor(frame, cv.COLOR_RGB2GRAY)
            )

            # to np
            frame = img_to_array(frame) * self.rescale

            # keep frame
            frames.append(frame)

In [7]:
# Copy of required pip import which no longer works in the package

"""
Utils module to provide some nice functions to develop with Keras and
Video sequences.
"""

import random as rnd
import secrets

import matplotlib.pyplot as plt
import numpy as np


def show_sample(g, index=0, random=False, row_width=22, row_height=5):
    """ Displays a batch using matplotlib.

    params:

    - g: keras video generator
    - index: integer index of batch to see (overriden if random is True)
    - random: boolean, if True, take a random batch from the generator
    - row_width: integer to give the figure height
    - row_height: integer that represents one line of image, it is multiplied by \
    the number of sample in batch.
    """
    total = len(g)
    if random:
        sample = secrets.randbelow(total)
    else:
        sample = index

    assert index < len(g)
    sample = g[sample]
    sequences = sample[0]
    labels = sample[1]

    rows = len(sequences)
    index = 1
    plt.figure(figsize=(row_width, row_height * rows))
    for batchid, sequence in enumerate(sequences):
        classid = np.argmax(labels[batchid])
        classname = g.classes[classid]
        cols = len(sequence)
        for image in sequence:
            plt.subplot(rows, cols, index)
            plt.title(classname)
            plt.imshow(image)
            plt.axis("off")
            index += 1
    plt.show()

In [8]:
import os
import glob
import keras

# Set up global params
SIZE = (500, 500)
CHANNELS = 3
NBFRAME = 6 # How many frames are to be extracted from each clip to represent a window
BS = 3 # How many windows are batched together into a training/validation batch?

# Explore it based on a limited set of classes
classes = ['TACKLE', 'LINEOUT', 'SCRUM']
classes.sort()

# pattern to get videos and classes
glob_pattern= save_path+'{classname}*.mp4'

t_files = glob.glob(save_path+'TACKLE*.mp4')

print(len(t_files))

# for data augmentation
data_aug = keras.preprocessing.image.ImageDataGenerator(
    zoom_range=0.1,
    horizontal_flip=True)

# Create video frame generator
train = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern,
    nb_frames=NBFRAME,
    split_test=0.01, 
    split_val=0.25, 
    shuffle=True,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    use_frame_cache=True)

# Get a validation data generator
valid = train.get_validation_generator()

183
class LINEOUT, validation count: 5, test count: 0, train count: 16
class SCRUM, validation count: 5, test count: 0, train count: 15
class TACKLE, validation count: 45, test count: 1, train count: 137
Total data: 3 classes for 168 files for train
Total data: 3 classes for 55 files for validation


In [9]:
import os
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv3D, MaxPooling3D, Dropout, BatchNormalization
from keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import h5py
import tensorflow as tf
from keras import layers
import einops

In [10]:
# Define the dimensions of one frame in the set of frames created
HEIGHT = 500
WIDTH = 500

In [11]:
class Conv2Plus1D(keras.layers.Layer):
  def __init__(self, filters, kernel_size, padding):
    """
      A sequence of convolutional layers that first apply the convolution operation over the
      spatial dimensions, and then the temporal dimension. 
    """
    super().__init__()
    self.seq = keras.Sequential([  
        # Spatial decomposition
        layers.Conv3D(filters=filters,
                      kernel_size=(1, kernel_size[1], kernel_size[2]),
                      padding=padding),
        # Temporal decomposition
        layers.Conv3D(filters=filters, 
                      kernel_size=(kernel_size[0], 1, 1),
                      padding=padding)
        ])

  def call(self, x):
    return self.seq(x)

In [12]:
class ResidualMain(keras.layers.Layer):
  """
    Residual block of the model with convolution, layer normalization, and the
    activation function, ReLU.
  """
  def __init__(self, filters, kernel_size):
    super().__init__()
    self.seq = keras.Sequential([
        Conv2Plus1D(filters=filters,
                    kernel_size=kernel_size,
                    padding='same'),
        layers.LayerNormalization(),
        layers.ReLU(),
        Conv2Plus1D(filters=filters, 
                    kernel_size=kernel_size,
                    padding='same'),
        layers.LayerNormalization()
    ])

  def call(self, x):
    return self.seq(x)

In [13]:
class Project(keras.layers.Layer):
  """
    Project certain dimensions of the tensor as the data is passed through different 
    sized filters and downsampled. 
  """
  def __init__(self, units):
    super().__init__()
    self.seq = keras.Sequential([
        layers.Dense(units),
        layers.LayerNormalization()
    ])

  def call(self, x):
    return self.seq(x)

In [14]:
def add_residual_block(input, filters, kernel_size):
  """
    Add residual blocks to the model. If the last dimensions of the input data
    and filter size does not match, project it such that last dimension matches.
  """
  out = ResidualMain(filters, 
                     kernel_size)(input)

  res = input
  # Using the Keras functional APIs, project the last dimension of the tensor to
  # match the new filter size
  if out.shape[-1] != input.shape[-1]:
    res = Project(out.shape[-1])(res)

  return layers.add([res, out])

In [15]:
class ResizeVideo(keras.layers.Layer):
  def __init__(self, height, width):
    super().__init__()
    self.height = height
    self.width = width
    self.resizing_layer = layers.Resizing(self.height, self.width)

  def call(self, video):
    """
      Use the einops library to resize the tensor.  

      Args:
        video: Tensor representation of the video, in the form of a set of frames.

      Return:
        A downsampled size of the video according to the new height and width it should be resized to.
    """
    # b stands for batch size, t stands for time, h stands for height, 
    # w stands for width, and c stands for the number of channels.
    old_shape = einops.parse_shape(video, 'b t h w c')
    images = einops.rearrange(video, 'b t h w c -> (b t) h w c')
    images = self.resizing_layer(images)
    videos = einops.rearrange(
        images, '(b t) h w c -> b t h w c',
        t = old_shape['t'])
    return videos

In [16]:
input_shape = (None, NBFRAME, HEIGHT, WIDTH, 3)
input = layers.Input(shape=(input_shape[1:]))
x = input

x = Conv2Plus1D(filters=16, kernel_size=(3, 7, 7), padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = ResizeVideo(HEIGHT // 2, WIDTH // 2)(x)

# Block 1
x = add_residual_block(x, 16, (3, 3, 3))
x = ResizeVideo(HEIGHT // 4, WIDTH // 4)(x)

# Block 2
x = add_residual_block(x, 32, (3, 3, 3))
x = ResizeVideo(HEIGHT // 8, WIDTH // 8)(x)

# Block 3
x = add_residual_block(x, 64, (3, 3, 3))
x = ResizeVideo(HEIGHT // 16, WIDTH // 16)(x)

# Block 4
x = add_residual_block(x, 128, (3, 3, 3))

x = layers.GlobalAveragePooling3D()(x)
x = layers.Flatten()(x)
x = layers.Dense(len(classes))(x)

model = keras.Model(input, x)

2023-02-27 13:05:45.118168: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-27 13:05:45.118384: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /dcs/19/u1901447/year4/project/ruck_and_roll/venv/lib/python3.9/site-packages/cv2/../../lib64:/local/java/postgresql/lib/:/local/java/postgresql/lib/
2023-02-27 13:05:45.120099: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /dcs/19/u1901447/year4/project/ruck_and_roll/venv/lib/python3.9/site-packages/cv2/../../lib64:/loca

In [17]:
# # Visualize the model
keras.utils.plot_model(model, expand_nested=True, dpi=60, show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [18]:
model.compile(loss = keras.losses.CategoricalCrossentropy(), 
              optimizer = keras.optimizers.Adam(learning_rate = 0.0001), 
              metrics = ['accuracy'])

In [19]:
history = model.fit(train, epochs = 10, validation_data = valid)

Epoch 1/10
56/56 [==============================] - 422s 7s/step - loss: 1.8039 - accuracy: 0.1131 - val_loss: 1.6500 - val_accuracy: 0.0926
Epoch 2/10
56/56 [==============================] - 401s 7s/step - loss: 1.6413 - accuracy: 0.0952 - val_loss: 1.6018 - val_accuracy: 0.0926
Epoch 3/10
56/56 [==============================] - 396s 7s/step - loss: 1.6186 - accuracy: 0.0952 - val_loss: 1.2730 - val_accuracy: 0.0741
Epoch 4/10
56/56 [==============================] - 403s 7s/step - loss: 1.5938 - accuracy: 0.0952 - val_loss: 1.5393 - val_accuracy: 0.0926
Epoch 5/10
56/56 [==============================] - 395s 7s/step - loss: 1.5643 - accuracy: 0.0952 - val_loss: 1.4946 - val_accuracy: 0.0926
Epoch 6/10
56/56 [==============================] - 400s 7s/step - loss: 1.5355 - accuracy: 0.0952 - val_loss: 1.4924 - val_accuracy: 0.0926
Epoch 7/10
56/56 [==============================] - 392s 7s/step - loss: 1.5351 - accuracy: 0.0952 - val_loss: 1.1939 - val_accuracy: 0.0741
Epoch 8/10
56

In [22]:
model.save(MODEL_LOCATION + 'my_model')

INFO:tensorflow:Assets written to: /dcs/large/u1901447/models/my_model/assets


INFO:tensorflow:Assets written to: /dcs/large/u1901447/models/my_model/assets


In [23]:
from sklearn.metrics import confusion_matrix, classification_report

y_prediction = model.predict(valid)
y_pred = np.argmax(y_prediction, axis=1)

print(y_pred)


# print('Confusion Matrix')
# print(confusion_matrix(valid.classes, y_pred))

print('Classification Report')
target_names = ['TACKLE', 'LINEOUT', 'SCRUM']
print(classification_report(valid.classes, y_pred, target_names=target_names))

18/18 [==============================] - 31s 2s/step
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Classification Report


ValueError: Found input variables with inconsistent numbers of samples: [3, 54]